### Attack the classifier of the last layer of Phi 3 model for one prompt at a time

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time
import pickle
from custom_model import CustomModel
from opt_utils import get_nonascii_toks, token_gradients, sample_control, get_filtered_cands, get_logits, load_model_and_tokenizer, get_prompt, get_primary_activation
from suffix_manager import SuffixManager

np.set_printoptions(suppress=True, linewidth=10000)
torch.set_printoptions(sci_mode=False, linewidth=100000, threshold=float('inf'))

In [3]:
model_path = 'loaded_models/phi3'

In [4]:
adv_string_init = "! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !"
model_id = "microsoft/Phi-3-mini-4k-instruct"

prompt_index = 0

num_steps = 100
topk = 64
batch_size = 64
allow_non_ascii = False  # you can set this to True to use unicode tokens

In [5]:
torch.cuda.empty_cache()

In [6]:
# Get the text from the test dataset
text = get_prompt(prompt_index)

In [ ]:
model, tokenizer = load_model_and_tokenizer(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
print(model.device)

cuda:0


In [8]:
suffix_manager = SuffixManager(tokenizer, text, adv_string_init)

In [ ]:
linear_model = pickle.load(open('./Task Drift/trained_linear_probes_microsoft/phi3/31/model.pickle', 'rb'))

custom_model = CustomModel(model, linear_model)
device = custom_model.base_model.get_input_embeddings().weight.device

In [10]:
print(device)

cuda:0


In [11]:
primary_activation = get_primary_activation(prompt_index)
primary_activation = primary_activation.to(device)

In [12]:
target = torch.tensor([0], dtype=torch.float32)
target = target.to(device)

In [13]:
def check_misclassification(adv_suffix):
    device = custom_model.base_model.get_input_embeddings().weight.device
    input_ids = suffix_manager.get_input_ids(adv_suffix).unsqueeze(0)
    input_ids = input_ids.to(device)

    attention_mask = torch.ones_like(input_ids).to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
    last_token_activation = outputs['hidden_states'][-1][0, -1].float()  # poisoned activation with suffix

    poisoned_activation = last_token_activation

    delta = (poisoned_activation - primary_activation).detach().cpu().float().numpy().reshape(1, -1)

    label = linear_model.predict(delta)
    prob = linear_model.predict_proba(delta)
    return label, prob

In [ ]:
not_allowed_tokens = None if allow_non_ascii else get_nonascii_toks(tokenizer)
adv_suffix = adv_string_init

start_time = time.time()

for i in range(num_steps + 1):

    # Step 1. Encode user prompt (behavior + adv suffix) as tokens and return token ids.
    input_ids = suffix_manager.get_input_ids(adv_suffix)
    input_ids = input_ids.to(device)

    # Step 2. Compute Coordinate Gradient
    coordinate_grad, loss, outputs, one_hot = token_gradients(custom_model, input_ids, suffix_manager.adv_string_slice, target, primary_activation)
    label, prob = check_misclassification(adv_suffix)
    print(f"i: {i}  loss: {loss:.10f}  {label}   {prob[0]}")
    print(adv_suffix)
    print("-------------------------------------------------------\n")

    if i == num_steps:
        break

    # Step 3. Sample a batch of new tokens based on the coordinate gradient.
    # Notice that we only need the one that minimizes the loss.
    with torch.no_grad():

        # Step 3.1 Slice the input to locate the adversarial suffix.
        adv_suffix_tokens = input_ids[suffix_manager.adv_string_slice].to(device)

        # Step 3.2 Randomly sample a batch of replacements.
        # Encoded suffixes
        new_adv_suffix_tokens = sample_control(
            adv_suffix_tokens,
            coordinate_grad,
            batch_size,
            topk=topk
        )

        # Step 3.3 This step ensures all adversarial candidates have the same number of tokens.
        # Decoded suffixes
        new_adv_suffix = get_filtered_cands(
            tokenizer,
            new_adv_suffix_tokens,
            filter_cand=True,
            curr_control=adv_suffix
        )

        # Step 3.4 Compute loss on these candidates and take the argmin.
        logits = get_logits(
            custom_model=custom_model,
            tokenizer=tokenizer,
            input_ids=input_ids,
            control_slice=suffix_manager.adv_string_slice,
            primary_activation=primary_activation,
            test_controls=new_adv_suffix,
            return_ids=False,
            batch_size=12 # decrease this number if you run into OOM.
        )
        target = target.to(logits.dtype)
        expanded_target = target.expand_as(logits)

        losses = nn.BCEWithLogitsLoss(reduction='none')(logits, expanded_target)

        best_new_adv_suffix_id = losses.argmin()
        best_new_adv_suffix = new_adv_suffix[best_new_adv_suffix_id]

        adv_suffix = best_new_adv_suffix

end_time = time.time()

print(f"Total time: {end_time - start_time} seconds")